<a href="https://colab.research.google.com/github/Giovanni-Ornellas/IA-de-Linguagem-Neutra/blob/main/IA_de_linguagem_Neutra_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q spacy PyPDF2 pandas
!python -m spacy download pt_core_news_lg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 2.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from google.colab import files
import spacy
from PyPDF2 import PdfReader
import pandas as pd
import re

In [ ]:
uploaded = files.upload()             # escolhe o PDF
pdf_path = list(uploaded.keys())[0]


def pdf_to_csv_local(pdf_path: str, csv_path: str = "/content/tokens.csv", model_name: str = "pt_core_news_lg"):
    # extrai texto
    reader = PdfReader(pdf_path)
    pages = [
        (i+1, page.extract_text())
        for i, page in enumerate(reader.pages)
        if page.extract_text()
    ]
    # carrega spaCy
    nlp = spacy.load(model_name)

    rows = []
    for page_num, text in pages:
        doc = nlp(text)
        for sent_id, sent in enumerate(doc.sents, start=1):
            for token in sent:
                row = {
                    "page":       page_num,
                    "sent_id":    sent_id,
                    "token_text": token.text,
                    "lemma":      token.lemma_,
                    "pos":        token.pos_,
                    "tag":        token.tag_,
                    "dep":        token.dep_,
                    "ent_iob":    token.ent_iob_,
                    "ent_type":   token.ent_type_,
                    "is_stop":    token.is_stop,
                }
                # expande feats morfológicas em colunas separadas
                for feat in str(token.morph).split("|"):
                    if "=" in feat:
                        k, v = feat.split("=", 1)
                        row[k] = v
                rows.append(row)

    df = pd.DataFrame(rows)
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"✅ CSV gerado em: {csv_path} ({len(df)} tokens)")
    return df

df = pdf_to_csv_local(pdf_path)
df.head()


Saving Dom_Casmurro-Machado_de_Assis.pdf to Dom_Casmurro-Machado_de_Assis (1).pdf
✅ CSV gerado em: /content/tokens.csv (90731 tokens)


,page,sent_id,token_text,lemma,pos,tag,dep,ent_iob,ent_type,is_stop,...,Definite,PronType,NumType,VerbForm,Voice,Mood,Person,Tense,Polarity,Case
0,1,1,Prefácio,prefácio,PROPN,PROPN,ROOT,O,,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,de,de,ADP,ADP,case,O,,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,Ana,Ana,PROPN,PROPN,nmod,B,PER,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,Maria,Maria,PROPN,PROPN,flat:name,I,PER,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,Haddad,Haddad,PROPN,PROPN,flat:name,I,PER,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = pd.read_csv('/content/tokens.csv', encoding='utf-8-sig')

tradutor_neutro = {
    r"\bele(s)?\b": "elu",
    r"\bela(s)?\b": "elu",
    r"\bdele(s)?\b": "delu",
    r"\bdela(s)?\b": "delu",
    r"\bseu(s)?\b": "seu",
    r"\bsua(s)?\b": "seu",
    r"\bo(s)?\b": "le",
    r"\ba(s)?\b": "le",
    r"\bum(s)?\b": "une",
    r"\buma(s)?\b": "une",
    r"\baluno(s)?\b": "estudante",
    r"\baluna(s)?\b": "estudante",
    r"\bprofessor(es)?\b": "docente",
    r"\bprofessora(s)?\b": "docente",
    r"\bmenino(s)?\b": "criança",
    r"\bmenina(s)?\b": "criança",
    r"\bgaroto(s)?\b": "jovem",
    r"\bgarota(s)?\b": "jovem",
    r"\btodos\b": "todes",
    r"\btodas\b": "todes",
    r"\bbem-vindo(s)?\b": "bem-vinde",
    r"\bbem-vinda(s)?\b": "bem-vinde",
    r"\bquerido(s)?\b": "queride",
    r"\bquerida(s)?\b": "queride",
    r"\bobrigado\b": "obrigade",
    r"\bobrigada\b": "obrigade",
    r"\bsenhor(es)?\b": "pessoa",
    r"\bsenhora(s)?\b": "pessoa",
    r"\bmoço(s)?\b": "pessoa",
    r"\bmoça(s)?\b": "pessoa",
}

patterns = [
    (re.compile(pat, flags=re.IGNORECASE), repl)
    for pat, repl in tradutor_neutro.items()
]

def neutralize_token(tok: str) -> str:
    out = tok
    for pat, repl in patterns:
        if pat.search(out):
            out = pat.sub(repl, out)
    # Preserva maiúscula inicial, se houver
    if tok and tok[0].isupper():
        out = out.capitalize()
    return out

df['token_neutral'] = df['token_text'].apply(neutralize_token)

df.to_csv('/content/tokens_neutro.csv', index=False, encoding='utf-8-sig')
print("✅ tokens_neutro.csv gerado com", len(df), "linhas.")


✅ tokens_neutro.csv gerado com 90731 linhas.


In [ ]:
df = pd.read_csv('/content/tokens_neutro.csv', encoding='utf-8-sig')

df = df[~df['pos'].isin(['PUNCT', 'SPACE'])].copy()

df['orig_tok']    = df['token_text'].str.lower()
df['neutro_tok']  = df['token_neutral'].str.lower()

df = df[df['orig_tok'].str.match(r'^[a-zçáéíóúâêôãõü]+$', na=False)]

df_sent = (
    df
    .groupby(['page','sent_id'])[['orig_tok','neutro_tok']]
    .agg(lambda toks: ' '.join(toks))
    .reset_index()
    .rename(columns={'orig_tok':'orig_sent','neutro_tok':'neutro_sent'})
)

df_sent = df_sent.dropna().drop_duplicates(subset=['orig_sent'])

train = df_sent.sample(frac=0.9, random_state=42)
valid = df_sent.drop(train.index)

train[['orig_sent','neutro_sent']].to_csv('/content/train.csv', index=False)
valid[['orig_sent','neutro_sent']].to_csv('/content/valid.csv', index=False)

print(f"✔️ Treino: {len(train)} sentenças, Validação: {len(valid)} sentenças")


✔️ Treino: 4223 sentenças, Validação: 469 sentenças
